## ex07 - Agrégation des données avec GROUP BY et ORDER BY

La clause ***GROUP BY*** est une clause facultative de l'instruction SELECT. 
La clause GROUP BY un groupe sélectionné de lignes en lignes récapitulatives par valeurs d'une ou plusieurs colonnes.

La clause GROUP BY renvoie une ligne pour chaque groupe. 

Pour chaque groupe, vous pouvez appliquer une fonction d'agrégation telle que ***MIN, MAX, SUM, COUNT ou AVG*** pour fournir plus d'informations sur chaque groupe.

In [1]:
%load_ext sql

### 0. Reconnectons notre Google Drive

In [2]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


### 1. Connexion à la Database fournie pour le TP : demo.db3

In [3]:
%sql sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3

'Connected: @/content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3'

Si vous ne connaissez toujours pas les tables présentes dans la database de démonstration :-), vous pouvez toujours utiliser la commande suivante pour le savoir. 

In [4]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. Regroupement des données

Prenons la table ***rch*** ici dans notre comme exemple

#### 2.1 Regardons tout d'abord les colonnes de la table


In [5]:
%sql SELECT * From rch LIMIT 3

 * sqlite:////content/gdrive/MyDrive/Partage/Notebooks_Serie_1/demo.db3
Done.


RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,EVAPcms,TLOSScms,SED_INtons,SED_OUTtons,SEDCONCmg_kg,ORGN_INkg,ORGN_OUTkg,ORGP_INkg,ORGP_OUTkg,NO3_INkg,NO3_OUTkg,NH4_INkg,NH4_OUTkg,NO2_INkg,NO2_OUTkg,MINP_INkg,MINP_OUTkg,CHLA_INkg,CHLA_OUTkg,CBOD_INkg,CBOD_OUTkg,DISOX_INkg,DISOX_OUTkg,SOLPST_INmg,SOLPST_OUTmg,SORPST_INmg,SORPST_OUTmg,REACTPSTmg,VOLPSTmg,SETTLPSTmg,RESUSP_PSTmg,DIFFUSEPSTmg,REACBEDPSTmg,BURYPSTmg,BED_PSTmg,BACTP_OUTct,BACTLP_OUTct,CMETAL_1kg,CMETAL_2kg,CMETAL_3kg,TOT_Nkg,TOT_Pkg,NO3ConcMg_l,WTMPdegc
1,1981,1,146.34376525878906,146.2524871826172,0.09128088504076004,0.0,2.3320461650655488e-07,61619.46484375,155.3719024658203,0.016086287796497345,0.0,0.04825887829065323,0.0,362.0486755371094,361.8135070800781,203.620849609375,421.18377685546875,0.0,23.0184326171875,0.016107244417071342,0.0,1.1839052307949238e-11,0.0,0.0,0.0,5627225.0,5623486.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,806.0157470703125,0.0,0.0,0.0
2,1981,1,96.22569274902344,96.18285369873047,0.042821284383535385,0.0,1.6426764659627224e-07,0.0,0.0,0.01363153662532568,0.0,0.04089462012052536,0.0,315.60052490234375,315.4579772949219,0.0,127.00502014160156,0.0,0.0,0.013656049966812134,0.0,4.136972177163251e-16,0.0,0.0,0.0,3757606.5,3698301.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,442.4630126953125,0.0,0.0,0.0
3,1981,1,11.952718734741211,11.861368179321289,0.09135182946920395,0.0,2.0325823868461157e-07,2.0325823868461157e-07,6.595060941805286e-09,0.01146628800779581,0.0,0.03439886122941971,0.009118038229644299,48.2963752746582,47.93150329589844,0.0,62.467620849609375,0.0,0.0,0.011485105380415916,0.0,5.941028783295818e-14,0.0,0.0,0.0,360979.90625,456115.90625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.39912414550781,0.009118038229644299,0.0,0.0


#### 2.2 Vérifier les valeurs uniques (distinctes)

Tout d'abord, regardons le nombre de valeurs distinctes que l'on trouve dans la colonne RCH de la table rch. 

Nous pouvons utiliser le mot clé ***DISTINCT*** conjointement avec l'instruction SELECT, afin d'éliminer tous les enregistrements en double et récupérer uniquement les enregistrements uniques.


In [6]:
%%sql sqlite://
SELECT COUNT(DISTINCT RCH) AS nRCH
FROM rch

Done.


nRCH
23


Nous pourrions également utiliser la clause ***GROUP BY*** afin d'obtenir une sortie plus lisible que la colonne de la table originelle car comportant moins de lignes : seules les valeurs uniques seront renvoyées.

In [7]:
%%sql sqlite://
SELECT RCH
FROM rch
GROUP BY RCH

Done.


RCH
1
2
3
4
5
6
7
8
9
10


#### 2.3 Utiliser les fonctions d'agrégation sur les groupes

Nous pouvons obtenir davanatges de détails en agrégeant des données sur un groupe plutôt que sur des colonnes entières.


In [8]:
%%sql sqlite://
SELECT RCH, AVG(FLOW_INcms), AVG(FLOW_OUTcms)
FROM rch
GROUP BY RCH

Done.


RCH,AVG(FLOW_INcms),AVG(FLOW_OUTcms)
1,631.8990093310674,629.7380514648225
2,333.0202187087801,332.79249324997267
3,152.36708157062532,151.58638913697666
4,270.92289432916374,270.5829954958624
5,933.1512839211358,921.7582115385268
6,2700.088200208876,2697.8029292848373
7,69.64315097878377,68.53229562627091
8,1784.046529504988,1776.2703920046488
9,1689.8438427501255,1688.7743001672957
10,233.96010857224465,233.50774793492423


### 3. Ordonner / Trier les enregistrements

Tout d'abord, vérifions les années et les mois avec le maximum FLOW_INcms


In [ ]:
%%sql sqlite://
SELECT RCH, YR, MO, MAX(FLOW_INcms)
FROM rch
GROUP BY RCH

Done.


RCH,YR,MO,MAX(FLOW_INcms)
1,1997,7,3758.95239258
2,1997,7,1820.23376465
3,1981,5,871.117736816
4,1997,7,1919.54003906
5,1997,7,3961.94506836
6,1992,6,10503.7236328
7,1992,3,352.160858154
8,1994,5,7693.54345703
9,1994,5,7507.53515625
10,2010,5,1513.65100098


Il est évident que les colonnes ***YR*** "année" et ***MO*** "mois" ne sont pas triées dans un ordre normal. 

C'est le bon moment pour utiliser la clause ***ORDER BY***, que vous pouvez placer à la fin d'une instruction SQL (après un WHERE et un GROUP BY). 

Nous pouvons trier les résultats de la requête par année, puis par mois.


In [9]:
%%sql sqlite://
SELECT RCH, YR, MO, ROUND(MAX(FLOW_INcms),2)
FROM rch
GROUP BY RCH
ORDER BY YR, MO

Done.


RCH,YR,MO,"ROUND(MAX(FLOW_INcms),2)"
3,1981,5,871.12
7,1992,3,352.16
6,1992,6,10503.72
8,1994,5,7693.54
9,1994,5,7507.54
11,1994,5,7375.6
17,1995,6,226.18
20,1996,6,1081.86
13,1997,6,3291.48
14,1997,6,1798.1


Par défaut, le tri se fait avec le paramètre ***ASC***, qui classe donc les données par ordre croissant. 

Nous pouvons trier par ordre décroissant en appliquant l'option ***DESC***.


In [10]:
%%sql sqlite://
SELECT RCH, YR, MO, ROUND(MAX(FLOW_INcms),2)
FROM rch
GROUP BY RCH
ORDER BY YR DESC, MO

Done.


RCH,YR,MO,"ROUND(MAX(FLOW_INcms),2)"
15,2010,2,445.3
10,2010,5,1513.65
12,2010,5,3167.54
16,1999,5,1628.88
13,1997,6,3291.48
14,1997,6,1798.1
18,1997,6,1161.75
19,1997,6,637.02
21,1997,6,590.98
22,1997,6,1428.44


### 4. Filtrer les données sur les groupes avec la clause HAVING

Parfois, nous souhaitons filtrer les enregistrements en fonction d'un groupe ou d'une valeur agrégée. 

Alors que le premier réflexe serait peut être d'utiliser une instruction WHERE, cela ne fonctionnera pas car le WHERE filtre les enregistrements et ne filtre pas les agrégations. 

Par exemple, nous essayons d'utiliser un WHERE pour filtrer les résultats où MAX (FLOW_INcms) est supérieur à 3000. 

Cela obtiendra une sqlite3.OperationalError pour cause de mauvaise utilisation de la fonction d'agrégation.

In [ ]:
%%sql sqlite://
SELECT RCH, YR, MO, MAX(FLOW_INcms) as MAX_FLOWIN
FROM rch
WHERE MAX_FLOWIN > 3000.0
GROUP BY RCH
ORDER BY YR DESC, MO

(sqlite3.OperationalError) misuse of aggregate: MAX() [SQL: u'SELECT RCH, YR, MO, MAX(FLOW_INcms) as MAX_FLOWIN\nFROM rch\nWHERE MAX_FLOWIN > 3000.0\nGROUP BY RCH\nORDER BY YR DESC, MO']


Dans ce cas, nous utiliserons la clause ***HAVING*** pour spécifier une condition de filtre pour un groupe ou un agrégat. 

La clause HAVING est une clause facultative de l'instruction SELECT. 

Nous utilisons souvent la clause HAVING avec la clause GROUP BY. 

La clause GROUP BY regroupe un ensemble de lignes en un ensemble de lignes ou de groupes récapitulatifs. 

La clause HAVING filtre ensuite les groupes en fonction des conditions spécifiées.

*** Il est à noter que la clause HAVING doit suivre strictement la clause GROUP BY. ***


In [ ]:
%%sql sqlite://
SELECT RCH, YR, MO, MAX(FLOW_INcms) as MAX_FLOWIN
FROM rch
GROUP BY RCH
HAVING MAX_FLOWIN > 3000.0
ORDER BY YR DESC, MO

Done.


RCH,YR,MO,MAX_FLOWIN
12,2010,5,3167.54492188
13,1997,6,3291.47900391
1,1997,7,3758.95239258
5,1997,7,3961.94506836
8,1994,5,7693.54345703
9,1994,5,7507.53515625
11,1994,5,7375.59814453
6,1992,6,10503.7236328


### Conclusion du notebook 

Dans ce TP, nous avons appris à utiliser l'opérateur DISTINCT afin d'obtenir des résultats distincts dans nos requêtes, et ainsi éliminer les doublons.

Ensuite, nous avons appris comment agréger et trier les données à l'aide de GROUP BY et ORDER BY.

Nous avons également montré la puissance des fonctions d'agrégation de SUM(), MAX(), MIN(), AVG() et COUNT().

De plus, nous avons utilisé la clause HAVING pour filtrer les champs agrégés, ce qui ne peut pas être traité avec la clause WHERE.
